In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json
from pprint import pprint

boto_session = Session()
region = boto_session.region_name


agent_name = "deep_research_scoping_agent"
agent_core_client = boto3.client('bedrock-agentcore-control', region_name='us-west-2')

## Update SSM Parameter store from .env

In [ ]:
# Load .env file and create AWS Systems Manager secure parameters
from pathlib import Path
import dotenv

env_path = Path("/workspaces/agent_core_deep_research/.env")

if not env_path.exists():
    print(f"❌ .env file not found at {env_path}")
else:
    # Load environment variables
    env_vars = dotenv.dotenv_values(env_path)
    
    print(f"Found {len(env_vars)} environment variables in .env")
    
    # Create SSM client
    ssm_client = boto_session.client("ssm", region_name="us-west-2")
    
    # Create secure parameters for each key
    for key, value in env_vars.items():
        if not value or value.strip() == "":
            print(f"⚠️  Skipping empty value for {key}")
            continue
            
        parameter_name = f"/deep_research_scoping_agent/{key}"
        
        try:
            ssm_client.put_parameter(
                Name=parameter_name,
                Value=value,
                Type="SecureString",
                Overwrite=True,
                Description=f"Environment variable for deep_research_scoping_agent"
            )
            print(f"✅ Created/Updated secure parameter: {parameter_name}")
        except Exception as e:
            print(f"❌ Failed to create parameter {parameter_name}: {e}")
    
    print(f"\n✅ All environment variables stored as secure parameters in us-west-2")# Build Docker image and push to ECR


## Build and Push Docker to ECR (seems overwriten by Agent Toolkits Config)

In [ ]:


import subprocess

# Get AWS account ID
account_id = boto_session.client("sts").get_caller_identity()["Account"]

# ECR repo URI
ecr_repo = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{agent_name}"

# Create ECR repository if it doesn't exist
print(f"Creating ECR repository: {agent_name}...")
ecr_client = boto_session.client("ecr")
try:
    ecr_client.create_repository(
        repositoryName=agent_name,
        imageScanningConfiguration={'scanOnPush': True}
    )
    print(f"✅ ECR repository '{agent_name}' created successfully")
except ecr_client.exceptions.RepositoryAlreadyExistsException:
    print(f"ℹ️  ECR repository '{agent_name}' already exists")

# Authenticate Docker to ECR
print("Authenticating to ECR...")
login_password = subprocess.run(
    ["aws", "ecr", "get-login-password", "--region", region],
    capture_output=True,
    text=True,
    check=True
).stdout

subprocess.run(
    ["docker", "login", "--username", "AWS", "--password-stdin", f"{account_id}.dkr.ecr.{region}.amazonaws.com"],
    input=login_password,
    text=True,
    check=True
)

# Build Docker image (run from parent directory)
print(f"Building Docker image: {agent_name}...")
# subprocess.run(
#     ["docker", "build", "-f", "src/Dockerfile", "-t", agent_name, "."],
#     cwd="/workspaces/agent_core_deep_research",
#     check=True
# )

subprocess.run(
    ["docker", "build", "-t", agent_name, "."],
    check=True
)


# Tag Docker image
print(f"Tagging image for ECR: {ecr_repo}:latest...")
subprocess.run(
    ["docker", "tag", f"{agent_name}:latest", f"{ecr_repo}:latest"],
    check=True
)

# Push Docker image to ECR
print(f"Pushing image to ECR: {ecr_repo}:latest...")
subprocess.run(
    ["docker", "push", f"{ecr_repo}:latest"],
    check=True
)

print(f"✅ Docker image successfully pushed to {ecr_repo}:latest")

## Create a build_role for CodeBuild

In [ ]:

# Create CodeBuild execution role
import json

iam_client = boto_session.client("iam")
role_name = f"deep-research-codebuild-role-{region}"

# IAM trust policy for CodeBuild
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "codebuild.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# IAM inline policy for CodeBuild
codebuild_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "CloudWatchLogs",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": f"arn:aws:logs:{region}:{account_id}:log-group:/aws/codebuild/*"
        },
        {
            "Sid": "ECRAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchCheckLayerAvailability",
                "ecr:GetDownloadUrlForLayer",
                "ecr:BatchGetImage",
                "ecr:GetAuthorizationToken",
                "ecr:PutImage",
                "ecr:InitiateLayerUpload",
                "ecr:UploadLayerPart",
                "ecr:CompleteLayerUpload"
            ],
            "Resource": [
                f"arn:aws:ecr:{region}:{account_id}:repository/{agent_name}",
                "*"
            ]
        },
        {
            "Sid": "S3SourceAccess",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:GetObjectVersion"
            ],
            "Resource": "arn:aws:s3:::*/*"
        }
    ]
}

# Create role
try:
    print(f"Creating IAM role: {role_name}...")
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="CodeBuild execution role for Bedrock AgentCore"
    )
    role_arn = create_role_response["Role"]["Arn"]
    print(f"✅ IAM role created: {role_arn}")
    
    # Attach inline policy
    print("Attaching inline policy...")
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName="CodeBuildPolicy",
        PolicyDocument=json.dumps(codebuild_policy)
    )
    print("✅ Inline policy attached successfully")
    
except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"ℹ️  IAM role '{role_name}' already exists")
    role_arn = f"arn:aws:iam::{account_id}:role/{role_name}"
    
    # Update inline policy if role exists
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName="CodeBuildPolicy",
            PolicyDocument=json.dumps(codebuild_policy)
        )
        print("✅ Inline policy updated successfully")
    except Exception as e:
        print(f"⚠️  Warning: Could not update policy: {e}")

print(f"\nCodeBuild Role ARN: {role_arn}")


## Config, Deploy and Launch Agent Core

In [ ]:
# we still need to configure but we'll use deployment_type="container"
# Since we already built and pushed the Docker image to ECR,
# and provide the ECR repository to skip the Dockerfile generation

agentcore_runtime = Runtime()

agent_name = "deep_research_scoping_agent"

# Get AWS account ID and construct ECR image URI
account_id = boto_session.client("sts").get_caller_identity()["Account"]
ecr_repository = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{agent_name}"

response = agentcore_runtime.configure(
    entrypoint="agent_core/scoping_agent_core.py",
    auto_create_execution_role=True,
    code_build_execution_role=role_arn,  # Use the role we created
    auto_create_ecr=False,  # Don't create new ECR - use existing
    deployment_type="container",  # Use container deployment
    region=region,
    agent_name=agent_name,
    requirements_file="requirements.txt",
    ecr_repository=ecr_repository  # Use the ECR image we built and pushed
)
response

##

In [ ]:
# Launch using the existing ECR image (skip the build process)
# Since we already built and pushed the image, we don't want to rebuild
launch_result = agentcore_runtime.launch()

## Grant Agent Core Execution Role permission to read SSM

In [ ]:
import boto3
from pprint import pprint

res = agent_core_client.list_agent_runtimes()
#pprint(res['agentRuntimes'])
agent_id = [item['agentRuntimeId'] for item in res['agentRuntimes'] if item['agentRuntimeName']==agent_name][0]

res = agent_core_client.get_agent_runtime(agentRuntimeId=agent_id)
execution_role_arn = res['roleArn']


In [ ]:
role_name = execution_role_arn.split('/')[-1]
ssm_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "SSMParameterStoreRead",
            "Effect": "Allow",
            "Action": [
                "ssm:GetParameter",
                "ssm:GetParameters",
                "ssm:GetParametersByPath"
            ],
            "Resource": f"arn:aws:ssm:us-west-2:482387069690:parameter/deep_research_scoping_agent/*"
        },
        {
            "Sid": "KMSDecrypt",
            "Effect": "Allow",
            "Action": ["kms:Decrypt"],
            "Resource": "*"
        }
    ]
}

iam_client = boto_session.client("iam")
iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName="SSMParameterStoreAccess",
    PolicyDocument=json.dumps(ssm_policy)
)

print(f"Updated role '{role_name}' with SSM Parameter Store access")


Updated role 'AmazonBedrockAgentCoreSDKRuntime-us-west-2-a59a4561a2' with SSM Parameter Store access


## Send payload

In [80]:
payload = {
    "prompt": "I want to research the best coffee shops in San Francisco.",
    "thread_id": "test-thread-1"
}
result = agentcore_runtime.invoke(payload)




NameError: name 'agentcore_runtime' is not defined

In [45]:
import json
json_response = json.loads(result['response'][0])
pprint(json_response)

{'messages': [{'additional_kwargs': {},
               'content': 'I want to research the best coffee shops in San '
                          'Francisco.',
               'id': 'b3643c6d-b3db-4819-a6b8-4be5c04dc335',
               'name': None,
               'response_metadata': {},
               'type': 'human'},
              {'additional_kwargs': {},
               'content': 'Could you please clarify what criteria are most '
                          "important to you when determining the 'best' coffee "
                          'shops? For example, are you interested in coffee '
                          'quality, ambiance, Wi-Fi availability, food '
                          'options, location, or something else?',
               'id': '65b5b390-bf3a-43bb-947a-f493bb5fca67',
               'invalid_tool_calls': [],
               'name': None,
               'response_metadata': {},
               'tool_calls': [],
               'type': 'ai',
               'usage_metadata'

In [ ]:
payload = {
    "prompt": "Let's focus on coffee quality",
    "thread_id": "test-thread-1"
}
result = agentcore_runtime.invoke(payload)

response_data = result['response']
if isinstance(response_data, str):
    # Single string response
    try:
        json_response = json.loads(response_data)
        print("\n=== Parsed JSON Response ===")
        pprint(json_response)
    except json.JSONDecodeError as e:
        print(f"\n⚠️  Response is not valid JSON: {e}")
        print("Raw response:", response_data)
elif isinstance(response_data, list):
    # List of response chunks - concatenate them
    full_response = ''.join(response_data)
    try:
        json_response = json.loads(full_response)
        print("\n=== Parsed JSON Response ===")
        pprint(json_response)
    except json.JSONDecodeError as e:
        print(f"\n⚠️  Concatenated response is not valid JSON: {e}")
        print("Full response length:", len(full_response))
        print("Full response:", full_response)

Result keys: dict_keys(['ResponseMetadata', 'runtimeSessionId', 'traceId', 'baggage', 'contentType', 'statusCode', 'response'])

Full result:
{'ResponseMetadata': {'HTTPHeaders': {'baggage': 'Self=1-6913d67b-0f731a9f3640e48d1b9ef569,session.id=b439b380-9960-475a-87a6-677a4764a6d8',
                                      'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Wed, 12 Nov 2025 00:36:18 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amzn-bedrock-agentcore-runtime-session-id': 'b439b380-9960-475a-87a6-677a4764a6d8',
                                      'x-amzn-requestid': '5bebd81a-406e-4aaa-8f9d-5b3960f5c9c2',
                                      'x-amzn-trace-id': 'Root=1-6913d67b-58e404b6085074362b22e91e;Parent=965f8fc63a761061;Sampled=1;Self=1-6913d67b-0f731a9f3640e48d1b9ef569'},
                      'HTTPS

In [63]:
result['response'][0]

# json_response = json.loads(result['response'][0])
# pprint(json_response)


'{"messages": [{"content": "I want to research the best coffee shops in San Francisco.", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": "b3643c6d-b3db-4819-a6b8-4be5c04dc335"}, {"content": "Could you please clarify what criteria are most important to you when determining the \'best\' coffee shops? For example, are you interested in coffee quality, ambiance, Wi-Fi availability, food options, location, or something else?", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": "65b5b390-bf3a-43bb-947a-f493bb5fca67", "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}, {"content": "Let\'s focus on coffee quality", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": "3aaee704-3b72-403e-b42e-867b73440931"}, {"content": "Thank you for clarifying that coffee quality is your primary criterion. I now have enough information to begin researching the best coffee shops in San Fra